Synthetic experiments for the paper: "On Lazy Training in Differentiable Programming"
--------------------

In [ ]:
using PyPlot, ProgressMeter
using Random, LinearAlgebra

Gradient descent for $2$-layers neural net (fixed training and test sets).

In [ ]:
"""
Gradient descent to train a 2-layers ReLU neural net for the square loss and with a scaling:
F(w) = MSE(scaling*f(w))/scaling^2
"""
function GDfor2NN(X_train, X_test, Y_train, Y_test, W_init, scaling, stepsize, niter) 
    (n,d) = size(X_train)
    m     = size(W_init, 1)
    W     = copy(W_init)   
    Ws   = zeros(m, d+1, niter)# store optimization path
    loss_train = zeros(niter)
    loss_test = zeros(niter)
    for iter = 1:niter
        Ws[:,:,iter] = W
        # output of the neural net
        temp    =  max.( W[:,1:end-1] * X_train', 0.0) # output hidden layer (size m × n)
        output  = scaling * sum( W[:,end] .* temp , dims=1) # output network (size 1 × n)
        # compute gradient
        gradR   = (output .- Y_train)'/n  # size n
        grad_w1 = (W[:,end] .* float.(temp .> 0) * ( X_train .* gradR )) # (size m × d) 
        grad_w2 = temp * gradR # size m
        grad = cat(grad_w1, grad_w2, dims=2) # size (m × d+1)   
        # store train loss
        loss_train[iter] = (1/2)*sum( ( output - Y_train).^2 )/n
        # store test loss  
        output = scaling .* sum( W[:,end] .* max.( W[:,1:end-1] * X_test', 0.0) , dims=1)
        loss_test[iter] = (1/2)*sum( ( output - Y_test).^2 )/length(Y_test)
        # gradient descent
        W = W - (stepsize/scaling) * grad
    end
    Ws, loss_train, loss_test
end

Stochastic gradient descent for $2$-layers neural net (directly minimizes the population loss)

In [ ]:
"""
SGD to train a 2-layers ReLU neural net of size m for the square loss and with a scaling:
F(w) = MSE(scaling*f(w))/scaling^2
teacher parameters are w0, θ0
"""
function populationSGDfor2NN(m, w0, θ0, stepsize, batchsize, scale, niter)  
    m0,d = size(θ0)
    θ = scale * randn(m,d) # start gradient flow with normalized data at fixed distance
    w = scale * randn(m,1)
    w[1:div(m,2)] = abs.(w[1:div(m,2)])
    w[(div(m,2)+1):end] = - abs.(w[1:div(m,2)])
    θ[(div(m,2)+1):end,:] = θ[1:div(m,2),:] # symmetrization

    θs = zeros(m,d,niter) # storing neurons
    ws = zeros(m,niter)   # storing output homogenizers
    val= zeros(niter,1)   # storing loss
    
    σ(x) = max(x,0) # ReLU activation
    σ′(x) = float(x>0)
    sigmaderinc(s) = float(s>0)

    # gradient flow
    for iter = 1:niter
        θs[:,:,iter] = θ
        ws[:,iter] = w
        # random data points
        X = randn(batchsize,d)
        X = X ./ sqrt.(sum(X.^2, dims=2))
        Y0 = sum( w0 .* σ.(θ0*X'), dims=1)/batchsize  #ground truth output
    
        # prediction and gradient computation
        temp = σ.( θ * X')    
        Y = sum( w .* temp, dims=1)/m
        val[iter] = (1/2)*sum( ( Y - Y0).^2 )/batchsize;
        gradR =  ( Y - Y0 )'/batchsize; # column of size batchsize
        gradw = temp * gradR
        gradθ = ((w.*sigmaderinc.(temp)) * ( X .* gradR ))#./(1+w)*2000
        

        θ = θ - stepsize * gradθ*d
        w = w - stepsize * gradw
    end
    ws,θs,val
end

### Illustration of the GD dynamics in 2-D

In [ ]:
Random.seed!(1);
include("illustration.jl")

#### Effect of the scale of initialization on generalization (GD)

In [ ]:
Random.seed!(1)
include("test_vs_scale.jl");

#### Effect of m on generalization, with two scalings (GD)

In [ ]:
Random.seed!(2)
include("test_vs_m.jl");

#### Effect of scaling with pure SGD

In [ ]:
Random.seed!(2)
include("populationSGD.jl");